In [1]:
import ujson as json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.plotly as py
import urllib2
import datetime
import sys

from moztelemetry.spark import get_pings, get_pings_properties
import moztelemetry.spark

%pylab inline

from operator import add

Unable to parse whitelist (/home/hadoop/anaconda2/lib/python2.7/site-packages/moztelemetry/histogram-whitelists.json). Assuming all histograms are acceptable.
Populating the interactive namespace from numpy and matplotlib


In [2]:
sc.defaultParallelism

256

### Extract a working dataset

Collect nightly data from builds dated within a 2-week window.

In [3]:
def fmt_date(d):
    return d.strftime("%Y%m%d")

## Dates bounding the time window to look at.
t1 = fmt_date(datetime.datetime.now() - datetime.timedelta(16)) # go back 16 days
t2 = fmt_date(datetime.datetime.now() - datetime.timedelta(2)) # go back 2 days
t1, t2

('20160831', '20160914')

In [4]:
## Collect only saved-session pings (which cover full browser sessions rather than the usual subsessions).

pings = get_pings(sc, app="Firefox", channel="nightly", build_id=(t1, t2), fraction=1.0)

In [5]:
def parseAddons(addons):
    """ Create a list of enabled add-ons with elements of the form (ID, version). """
    return[(k, v.get("version")) for k, v in addons.iteritems()]

def extract(ping):
    """ Extract relevant fields from each payload.
    
        shims: Reason why add-on shims were used, keyed by add-on ID (enumerated count of reason codes)
        cpowTime: Contiguous time spent by an add-on blocking the main loop by performing a blocking
                  cross-process call (microseconds, keyed by add-on ID).
        cpowForbidden: Number of times an add-on used CPOWs when it was marked as e10s compatible
                       (count, keyed by add-on ID).
        addons: List of (ID, version) for each enabled add-on.
    """
    hists = ping["payload"].get("histograms", {})
    keyed = ping["payload"].get("keyedHistograms", {})
    clientId = ping.get("clientId", None)
    return {
        "clientId": ping.get("clientId", None),
        "os": ping["environment"]["system"]["os"]["name"],
        "e10s": ping["environment"]["settings"]["e10sEnabled"],
        "sessionLength": ping["payload"]["info"]["sessionLength"], ## in seconds
        "shims": keyed.get("ADDON_SHIM_USAGE", {}),
        "cpowTime": keyed.get("PERF_MONITORING_SLOW_ADDON_CPOW_US", {}),
        "cpowForbidden": keyed.get("ADDON_FORBIDDEN_CPOW_USAGE", {}),
        "addons": parseAddons(ping["environment"].get("addons", {}).get("activeAddons", {}))
    }

## Extract relevant data, and restrict to clients that have add-ons.
bySession = pings.map(extract)\
    .filter(lambda p: p["addons"])\
    .persist(StorageLevel.MEMORY_AND_DISK_SER)

The `bySession` dataset has one record per client session which had enabled add-ons.

How many session pings are in the dataset?

In [6]:
bySession.count()

904188

How many unique clients do these come from?

In [7]:
bySession.map(lambda p: p["clientId"]).distinct().count()

53500

How many add-ons are represented in the dataset, and what are the top few?

In [8]:
addonCounts = bySession.flatMap(lambda p: [(guid, p["clientId"]) for (guid, version) in p["addons"]])\
    .distinct()\
    .map(lambda (guid, clientid): guid)\
    .countByValue()
len(addonCounts)

7268

In [9]:
sorted(addonCounts.items(), key = lambda (guid, count): (-count, guid))[:20]

[(u'flyweb@mozilla.org', 53352),
 (u'webcompat@mozilla.org', 53312),
 (u'e10srollout@mozilla.org', 53243),
 (u'firefox@getpocket.com', 53073),
 (u'{d10d0bf8-f5b5-c8b4-a8b2-2b9879e08c5d}', 10682),
 (u'uBlock0@raymondhill.net', 4534),
 (u'{b9db16a4-6edc-47ec-a1f4-b86292ed211d}', 2588),
 (u'{e4a8a97b-f2ed-450b-b12d-ee082ba24781}', 2435),
 (u'{DDC359D1-844A-42a7-9AA1-88A850A938A8}', 1701),
 (u'{73a6fe31-595d-460b-a920-fcc0f8843232}', 1598),
 (u'firebug@software.joehewitt.com', 1490),
 (u'{46551EC9-40F0-4e47-8E18-8E5CF550CFB8}', 1338),
 (u'firefox@mega.co.nz', 1297),
 (u'firefox@ghostery.com', 1230),
 (u'{b9bfaf1c-a63f-47cd-8b9a-29526ced9060}', 1153),
 (u'adbhelper@mozilla.org', 1133),
 (u'fxdevtools-adapters@mozilla.org', 1106),
 (u'loop@mozilla.org', 1074),
 (u'wrc@avast.com', 1019),
 (u'support@lastpass.com', 1017)]

Restrict computations to a set of add-ons (GUID/version pairs) that have enough data to draw reasonable measurements. We consider add-ons that are installed in at least 50 profiles, and have been active during a combined session time amounting to at least 1 hour in length.

In [10]:
## Get a count of unique profiles for each (GUID, version) pair.
addonInstalls = bySession.flatMap(lambda p: [(addon, p["clientId"]) for addon in p["addons"]])\
    .distinct()\
    .map(lambda (addon, clientId): (addon, 1))\
    .reduceByKey(add)

## Get total session time for each (GUID, version) pair.
addonTime = bySession.flatMap(lambda p: [(addon, p["sessionLength"]) for addon in p["addons"]])\
    .reduceByKey(add)

addonStats = addonInstalls.join(addonTime).\
    filter(lambda (addon, (nInstalls, totalTime)): nInstalls >= 50 and totalTime >= 3600)

How many add-ons (split by version) does this leave?

In [11]:
addonStats.count()

513

What are the top (GUID, version) pairs (by installs)?

In [12]:
addonStats.sortBy(lambda (addon, (nInstalls, totalTime)): -nInstalls).take(20)

[((u'flyweb@mozilla.org', u'1.0.0'), (53352, 7187932518)),
 ((u'webcompat@mozilla.org', u'1.0'), (53312, 7165476883)),
 ((u'firefox@getpocket.com', u'1.0.4'), (53072, 7139153099)),
 ((u'e10srollout@mozilla.org', u'1.2'), (51331, 6428335804)),
 ((u'e10srollout@mozilla.org', u'1.0'), (22719, 749662261)),
 ((u'{d10d0bf8-f5b5-c8b4-a8b2-2b9879e08c5d}', u'2.7.3'), (10436, 2013608779)),
 ((u'uBlock0@raymondhill.net', u'1.9.4'), (3593, 561556693)),
 ((u'uBlock0@raymondhill.net', u'1.9.6'), (3160, 435102454)),
 ((u'{b9db16a4-6edc-47ec-a1f4-b86292ed211d}', u'6.0.0'), (2472, 396076044)),
 ((u'{e4a8a97b-f2ed-450b-b12d-ee082ba24781}', u'3.9'), (2301, 628849576)),
 ((u'{DDC359D1-844A-42a7-9AA1-88A850A938A8}', u'3.0.6'), (1378, 236049213)),
 ((u'firebug@software.joehewitt.com', u'2.0.17'), (1358, 249053932)),
 ((u'{73a6fe31-595d-460b-a920-fcc0f8843232}', u'2.9.0.14'), (1295, 277290840)),
 ((u'{46551EC9-40F0-4e47-8E18-8E5CF550CFB8}', u'2.0.7'), (1271, 307393202)),
 ((u'firefox@ghostery.com', u'6.3.2')

### Shims

Count enabled add-on installs (ID and version), together with whether or not they were observed to use shims.

An add-on is counted as using shims if it has entry in the `ADDON_SHIM_USAGE` keyed histogram for at least one client session (regardless of the values in the histogram). This histogram records shim usage occurrence by the [reason it was used](https://dxr.mozilla.org/mozilla-central/source/toolkit/components/addoncompat/CompatWarning.jsm#94).

In [13]:
## Summarize shim usage per add-on/client.
## Reduce multiple sessions observed for each client to
## a single entry of the form ((ID, version), clientID, usedShims).

def getShimData(d):
    """ Summarize each add-on in the ping as (((GUID, version), clientID), <used shims?>). """
    return [((addonv, d["clientId"]), addonv[0] in d["shims"]) for addonv in d["addons"]]

shimUsageByClient = bySession.flatMap(getShimData)\
    .reduceByKey(lambda a, b: a or b)\
    .map(lambda ((addon, clientId), usedShims): (addon, clientId, usedShims))

In [14]:
## Compute number of clients that used shims for each add-on.
## Result is of the form ((ID, version), usedShims, # clients).

shimUsageCounts = shimUsageByClient\
    .map(lambda (addon, clientId, usedShims): ((addon, usedShims), 1))\
    .reduceByKey(add)\
    .map(lambda ((addon, usedShims), count): (addon, usedShims, count))

In [15]:
## For each (add-on, version) pair, determine whether it ever used shims,
## along with its overall installation count.
## Result is of the form
##  ((ID, version), <used shims in at least one client session>, overall # installations).

shimUsageByAddon = shimUsageCounts\
    .map(lambda (addon, usedShims, count): (addon, (usedShims, count)))\
    .reduceByKey(lambda (s1, c1), (s2, c2): (s1 or s2, c1 + c2))\
    .map(lambda (addon, (usedShims, count)): (addon, usedShims, count))

Sanity check: how many add-on (GUID, version) pairs do we have?

In [16]:
shimUsageByAddon.count()

9929

Restrict the final shim usage dataset to the set of add-ons we are interested in.

In [17]:
shimUsageFiltered = shimUsageByAddon\
    .map(lambda (addon, usedShims, count): (addon, (usedShims, count)))\
    .join(addonStats)\
    .map(lambda (addon, (shimData, stats)): (addon,) + shimData)\
    .collect()

## Order by decreasing installation count.
shimUsageFiltered.sort(key = lambda v: (-v[-1], v[:-1]))

How many add-ons are in the final dataset?

In [18]:
len(shimUsageFiltered)

513

How many of these used shims?

In [19]:
shimUsageShimmed = filter(lambda (addon, usedShims, count): usedShims, shimUsageFiltered)
len(shimUsageShimmed)

258

Dump results to a JSON file that will be used in the HTML page.

The file is one big JSON array with elements of the form `[<numInstallations>, [<GUID>, <version>], <usedShims>]`.

In [20]:
def formatForJSON(d):
    return (d[-1],) + d[:-1]

shimUsageOutput = map(formatForJSON, shimUsageFiltered)

try:
    output = open('output/shim-data.json', 'w')
    json.dump(shimUsageOutput, output)
    output.close()
except:
    pass

try:
    output = open('shim-data.json', 'w')
    json.dump(shimUsageOutput, output)
    output.close()
except:
    pass

The shim usage data, orderd by decreasing installation count.

In [21]:
shimUsageFiltered

[((u'flyweb@mozilla.org', u'1.0.0'), False, 53352),
 ((u'webcompat@mozilla.org', u'1.0'), False, 53312),
 ((u'firefox@getpocket.com', u'1.0.4'), True, 53072),
 ((u'e10srollout@mozilla.org', u'1.2'), False, 51331),
 ((u'e10srollout@mozilla.org', u'1.0'), False, 22719),
 ((u'{d10d0bf8-f5b5-c8b4-a8b2-2b9879e08c5d}', u'2.7.3'), True, 10436),
 ((u'uBlock0@raymondhill.net', u'1.9.4'), True, 3593),
 ((u'uBlock0@raymondhill.net', u'1.9.6'), True, 3160),
 ((u'{b9db16a4-6edc-47ec-a1f4-b86292ed211d}', u'6.0.0'), True, 2472),
 ((u'{e4a8a97b-f2ed-450b-b12d-ee082ba24781}', u'3.9'), True, 2301),
 ((u'{DDC359D1-844A-42a7-9AA1-88A850A938A8}', u'3.0.6'), False, 1378),
 ((u'firebug@software.joehewitt.com', u'2.0.17'), False, 1358),
 ((u'{73a6fe31-595d-460b-a920-fcc0f8843232}', u'2.9.0.14'), True, 1295),
 ((u'{46551EC9-40F0-4e47-8E18-8E5CF550CFB8}', u'2.0.7'), False, 1271),
 ((u'firefox@ghostery.com', u'6.3.2'), True, 1187),
 ((u'adbhelper@mozilla.org', u'0.8.7'), False, 1125),
 ((u'fxdevtools-adapters@mo

### CPOWs

CPOW usage is recorded in the `PERF_MONITORING_SLOW_ADDON_CPOW_US` histogram as time in microseconds spent by an add-on blocking the main loop using a CPOW.

Summarize CPOW usage for each enabled add-on (ID and version) by:

- average number of microseconds per CPOW blocking occurrence
- average number of blocking occurrences per hour of session time

Note that in many cases an add-on has an entry in the histogram, but it only has observations with the value 0. It appears that the histogram is automatically recorded at the same time as `PERF_MONITORING_SLOW_ADDON_JANK_US` by the [AddonWatcher](https://dxr.mozilla.org/mozilla-central/source/toolkit/components/perfmonitoring/AddonWatcher.jsm#128), and so may not have any CPOW blocking to report at that time. This is handled by dropping observations in the histograms' '0' bucket.

In [22]:
## Summarize CPOW usage per add-on, returning entries of the form
## ((GUID, version), { 
##    "totalTime" : <total session time in seconds with this add-on>,
##    "numOccurrences": <total number of times add-on CPOW blocked main loop>,
##    "totalCPOWTime": <total blocking time for add-on CPOWs>
## })

def getCPOWData(d):
    """ Summarize CPOW data for each add-on/session as a list of
            ((GUID, version), {totalTime, numOccurrences, totalCPOWTime}).
    """
    result = []
    for addonv in d["addons"]:
        data = {
            "totalTime": d["sessionLength"],
            "numOccurrences": 0,
            "totalCPOWTime": 0
        }
        cpowData = d["cpowTime"].get(addonv[0])
        
        ## If the histogram is present, but all values are 0, ignore it completely.
        if cpowData and cpowData["sum"] > 0:
            ## Some of the CPOW values may be 0 - ignore those.
            data["numOccurrences"] = sum([n for v, n in cpowData["values"].items() if v != "0"])
            data["totalCPOWTime"] = cpowData["sum"]
        result.append((addonv, data))
    return result

def dictSum(a, b):
    """ Add up like entries between two dicts. """
    result = {}
    for k in a:
        result[k] = a[k] + b[k]
    return result

cpowBySession = bySession.flatMap(getCPOWData).reduceByKey(dictSum)

Restrict the final CPOW dataset to the set of add-ons we are interested in.

In [23]:
cpowFiltered = cpowBySession\
    .join(addonStats)\
    .map(lambda (addon, (cpowData, stats)): (addon, cpowData))

In [24]:
## Summarize add-on CPOW usage with:
## - hadCPOWBlocking: were there any CPOW blocking occurrences?
## - avgBlockingTime: the average blocking time spent per occurrence (truncated to the nearest microsecond)
## - occurrenceFreq: the average number of blocking occurrences per session hour.

def summaryCPOWTime(d):
    sessionHours = float(d["totalTime"]) / 3600
    hadCPOWBlocking = d["numOccurrences"] > 0
    return {
        "hadCPOWBlocking": hadCPOWBlocking,
        ## Since these are microseconds anyway, truncate using integer division.
        "avgBlockingTime": d["totalCPOWTime"] / d["numOccurrences"] if hadCPOWBlocking else 0,
        "occurrenceFreq": float(d["numOccurrences"]) / sessionHours if hadCPOWBlocking else 0
    }

cpowSummary = cpowFiltered.mapValues(summaryCPOWTime).collect()

How many add-ons are left in the dataset?

In [25]:
len(cpowSummary)

513

How many of these had blocking CPOWs?

In [26]:
cpowHadBlocking = filter(lambda (addon, summary): summary["hadCPOWBlocking"], cpowSummary)
len(cpowHadBlocking)

275

Dump data for add-ons that had CPOW blocking to a JSON file that will be used in the HTML page.

The file is one big JSON array with elements of the form `[[<GUID>, <version>], <avgBlockingTime>, <occurrenceFreq>]`.

In [27]:
def formatForJSON((addon, summary)):
    return (addon, summary["avgBlockingTime"], summary["occurrenceFreq"])

## Order by decreasing average blocking time.
cpowOutput = map(formatForJSON, cpowSummary)
cpowOutput.sort(key = lambda (addon, avg, freq): -avg)


try:
    output = open('output/cpow-data.json', 'w')
    json.dump(cpowOutput, output)
    output.close()
except:
    pass

try:
    output = open('cpow-data.json', 'w')
    json.dump(cpowOutput, output)
    output.close()
except:
    pass

The blocking CPOW data, ordered by decreasing average blocking time.

In [28]:
sorted(cpowHadBlocking, key = lambda (addon, summary): summary["avgBlockingTime"], reverse = True)

[((u'{888d99e7-e8b5-46a3-851e-1ec45da1e644}', u'45.0.0'),
  {'avgBlockingTime': 3018815L,
   'hadCPOWBlocking': True,
   'occurrenceFreq': 0.2213008497860617}),
 ((u'{53A03D43-5363-4669-8190-99061B2DEBA5}', u'1.5.14'),
  {'avgBlockingTime': 2954625,
   'hadCPOWBlocking': True,
   'occurrenceFreq': 0.018699107996611057}),
 ((u'{a7c6cf7f-112c-4500-a7ea-39801a327e5f}', u'2.0.28'),
  {'avgBlockingTime': 1696027,
   'hadCPOWBlocking': True,
   'occurrenceFreq': 0.006282006370627532}),
 ((u'{02450914-cdd9-410f-b1da-db004e18c671}', u'0.99.06c'),
  {'avgBlockingTime': 950836,
   'hadCPOWBlocking': True,
   'occurrenceFreq': 0.04049731842080727}),
 ((u'{f36c6cd1-da73-491d-b290-8fc9115bfa55}',
   u'3.0.9.1-signed.1-let-fixed.1-signed'),
  {'avgBlockingTime': 681573L,
   'hadCPOWBlocking': True,
   'occurrenceFreq': 40.39545638495176}),
 ((u'omnibar@ajitk.com', u'0.7.28.20141004.1-signed.1-signed'),
  {'avgBlockingTime': 558370,
   'hadCPOWBlocking': True,
   'occurrenceFreq': 0.00752980767762149

The blocking CPOW data, ordered by decreasing occurrence frequency.

In [29]:
sorted(cpowHadBlocking, key = lambda (addon, summary): summary["occurrenceFreq"], reverse = True)

[((u'{4ED1F68A-5463-4931-9384-8FFF5ED91D92}', u'5.0.226.0'),
  {'avgBlockingTime': 159455L,
   'hadCPOWBlocking': True,
   'occurrenceFreq': 329.6188498994081}),
 ((u'thumbnailZoom@dadler.github.com', u'4.0'),
  {'avgBlockingTime': 156173L,
   'hadCPOWBlocking': True,
   'occurrenceFreq': 84.06607614679335}),
 ((u'paulsaintuzb@gmail.com', u'8.2.1'),
  {'avgBlockingTime': 311815L,
   'hadCPOWBlocking': True,
   'occurrenceFreq': 55.69460129134825}),
 ((u'{19503e42-ca3c-4c27-b1e2-9cdb2170ee34}', u'1.5.6.13'),
  {'avgBlockingTime': 310461L,
   'hadCPOWBlocking': True,
   'occurrenceFreq': 53.72607373684629}),
 ((u'artur.dubovoy@gmail.com', u'13.2.4'),
  {'avgBlockingTime': 299503L,
   'hadCPOWBlocking': True,
   'occurrenceFreq': 48.65258293460741}),
 ((u'{73a6fe31-595d-460b-a920-fcc0f8843232}', u'2.9.0.14rc1'),
  {'avgBlockingTime': 125677L,
   'hadCPOWBlocking': True,
   'occurrenceFreq': 45.89489410005144}),
 ((u'coba@mozilla.com.cn', u'1.0.33'),
  {'avgBlockingTime': 165089L,
   'hadC